# Функции в Pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
stats = pd.read_excel('../files/ad_campaigns.xlsx')
stats.head()

In [ ]:
stats.columns = ['group', 'phrase', 'effect', 'ad_id', 'title', 'text', 'link']
stats.head()

### Lambda-функции (Анонимные функции)
Хотим посчитать распределение количества слов в столбце с фразами

In [ ]:
stats['word_count'] = stats['phrase'].apply(lambda x: len(x.split(' ')))
stats.head()

In [ ]:
# вариант с передачей всей строчки функции
# тут надо обязательно указать параметр axis = 1

stats['word_count'] = stats.apply(lambda x: len(x['phrase'].split(' ')), axis=1)
stats.head()

In [ ]:
stats['word_count'].hist(bins=30)

### Упражнение
Поисковые запросы с каким количеством слов встречаются в наших данных чаще всего? Выведите топ-5

In [ ]:
stats[['phrase', 'word_count']].sort_values(by='word_count', ascending=False)

### Произвольные функции
В URL кампаний есть названия. С этим надо что-то делать

In [ ]:
# пример ссылки
url = stats.loc[0, 'link']
url

In [ ]:
from urllib import parse

In [ ]:
parsed = parse.urlsplit(url)
parsed

In [ ]:
# как доставать значения
parsed.netloc

In [ ]:
params = parse.parse_qs(parsed.query)
params

In [ ]:
# вот и кампании
params['utm_campaign'][0]

In [ ]:
# оборачиваем все в функцию
# в качестве аргумента будет строка датафрейма
def campaign_name(row):
    """Получение названия кампании из ссылки внутри строки row"""
    parsed = parse.urlsplit(row['link'])
    params_dict = parse.parse_qs(parsed.query)
    return params_dict['utm_campaign'][0]

In [ ]:
stats['campaign'] = stats.apply(campaign_name, axis=1)
stats.head()

In [ ]:
# как передать несколько аргументов
def power_up(row, n):
    """Возводит значение столбца effect в степень n"""
    return row['effect'] ** n

In [ ]:
stats['power_up'] = stats.apply(power_up, n=2, axis=1)
stats.head()

### Упражнение
В наших данных есть много объявлений с услугой МРТ (в названии группы есть слово 'мрт') круглосуточно (в тексте объявления есть '24 часа'). Выделите строки, в которых не упоминается МРТ, но прием идет круглосуточно. Сколько таких строк в датасете?

In [ ]:
filtration = stats['phrase'].str.contains('24 часа') & ~stats['phrase'].str.contains('мрт')

In [ ]:
stats[filtration]

# Группировки

In [ ]:
# подсчет количества записей по колонкам
stats['campaign'].value_counts().head()

In [ ]:
# нужно что-то более универсальное например аггрегированные табилицы
stats.groupby('campaign').count().head()

In [ ]:
# тоже самое, что с value_counts
stats.groupby('campaign').count().reset_index().sort_values('group', ascending=False).head() 

In [ ]:
# но теперь можно менять функции
stats.groupby('campaign').sum().sort_values('effect').head()

### Упражнение
Для каждой кампании campaign найдите минимальное значение столбца effect и постройте рейтинг кампаний по этим значениям, начиная с самого низкого.

In [ ]:
# задаем несколько функций сразу
stats.groupby('campaign').agg(['min', 'max'])['effect'].head()

In [ ]:
stats.head()

In [ ]:
# разные метрики для разных столбцов
stats.groupby('campaign').agg({'effect': ['min', 'max'], 'power_up': 'mean'}).head()

In [ ]:
# группировка по нескольким столбцам
stats.groupby(['group', 'campaign']).count().head()

In [ ]:
# максимальное число объявлений в одной группе

stats.groupby(['group', 'campaign']).count().sort_values('phrase', ascending=False).head()

### Упражнение
Какая кампания имеет наименьшую сумму по столбцу effect? 

# Агрегированные группировки
Считаем метрики для пользователей, которые выставили 100 и более оценок

In [ ]:
ratings = pd.read_csv('../files/ratings.csv')
ratings.head()

### Упражнение
Получите список пользователей film_fans_user_ids, которые выставили 100 и более оценок фильмам. Чтобы перевести pandas series в список можно использовать метод tolist():

film_fans_user_ids = some_dataframe['userId'].tolist()

In [ ]:
ratings_count = ratings.groupby('userId').count().reset_index()[['userId', 'movieId']]
ratings_count.head()

In [ ]:
film_fans_ratings_count = ratings_count[ ratings_count['movieId'] >= 100 ]
film_fans_ratings_count.head()

In [ ]:
film_fans_user_ids = film_fans_ratings_count['userId'].tolist()

In [ ]:
fans_data = ratings[ ratings['userId'].isin(film_fans_user_ids) ]
fans_data.head()

# Сводные таблицы
Прям как в экселе

In [ ]:
data = pd.read_csv('../files/ratings.csv')
data.head()

In [ ]:
# какие оценки поставил пользователь №1
data[ data['userId'] == 1 ]['rating'].value_counts()

In [ ]:
# а для всех пользователей сразу?
data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)

In [ ]:
# можно итоги добавить
data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0,
                 margins = True)

### Упражнение
Какой пользователь выставил больше всех пятерок?

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

# Немного про визуализации

### Matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('sales_data.csv', sep=';')
data.head()

In [ ]:
plt.plot(data['month'], data['2016'])
plt.plot(data['2017'])
plt.plot(data['2018'])

In [ ]:
plt.plot(data['month'], data['2016'], marker='o')
plt.plot(data['2017'], linestyle='dashed')
plt.plot(data['2016'], linewidth=5)

plt.legend()

### Seaborn
http://seaborn.pydata.org/examples/

In [ ]:
features = pd.read_csv('correlation.tsv', sep='\t')
features.head()

In [ ]:
# месяц сейчас не пригодится

features.drop('month', axis=1, inplace=True)
features.head()

In [ ]:
features.corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
f, ax = plt.subplots(figsize=(9, 6))

sns.heatmap(features.corr())

In [ ]:
# добавляем наглядности

f, ax = plt.subplots(figsize=(9, 6))

sns.heatmap(features.corr(), annot=True, fmt='.1f', ax=ax, cmap=sns.color_palette('coolwarm', 16))

### Plotly
Самая жесть

https://plot.ly/python/

In [ ]:
data = pd.read_csv('sales_data.csv', sep=';')
data.head()

In [ ]:
from yaml import load

In [ ]:
config = load(open('config.yaml'))

username = config['username']
api_key = config['api_key']

In [ ]:
import plotly
plotly.tools.set_credentials_file(username=username, api_key=api_key)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
trace = go.Table(

    header=dict(values=list(data.columns),

                fill = dict(color='#C2D4FF'),

                align = ['left', 'right', 'right', 'right']),

    cells=dict(values=[data['month'], data['2016'], data['2017'], data['2018']],

               fill = dict(color='#F5F8FF'),

               align = ['left'] + ['right']*3))

py.iplot([trace], filename = 'pandas_table')